# 4.2 범주형 데이터 다루기

4.1절에서는 수치형 데이터만 사용했지만, **실제 데이터 셋에는 범주형 데이터도 포함**되어 있다.<br>

범주형 데이터는 순서가 있는 것과 없는 것을 구분해야 한다.
- **순서가 있는 특성**: 정렬하거나 차례대로 놓을 수 있다.
      ex) '최상 > 상 > 중 > 하 > 최하'
- **순서가 없는 특성**: 차례를 부여할 수 없다.
      ex) 축구 팀 이름: 맨체스터 유나이티드, 첼시, 리버풀, 아스날
---

#### 예제를 위한 범주형 데이터가 포함된 DataFrame

새로운 DataFrame에는 다양한 특성을 가진 데이터가 준비되어 있다.<br>
- **순서가 없는 특성**: ```color```
- **순서가 있는 특성**: ```size```
- **수치형 특성**: ```price```
- **클래스 레이블**: ```classlabel```

In [25]:
import pandas as pd
df = pd.DataFrame([
                   ['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## 4.2.1 순서가 있는 특성 매핑

학습 알고리즘이 순서가 있는 특성을 제대로 인식하려면 범주형 문자열 값을 정수형으로 바꿔야 한다.<br>

                                          XL = L + 1 = M + 2

In [26]:
size_mapping = {
                'XL': 3,
                'L': 2,
                'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


---
만약 정수 값을 **다시 원래 문자열로 바꾸고 싶다면** 거꾸로 매핑하는 딕셔너리를 정의하면 된다.<br><br>

```inv_size_mapping = {v: k for k, v in size_mapping.items()}```

<br>

```size_mapping``` 딕셔너리와 비슷하게 **판다스의 map 메소드를 사용**하여 변환된 특성 열에 적용할 수 있다.

In [27]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## 4.2.2 클래스 레이블 인코딩

사이킷런은 대부분 클래스 레이블을 정수로 변환해 주지만, 실수를 방지하기 위해서 **클래스 레이블을 정수 배열로 전달하는 것이 좋다**.<br>

클래스 레이블은 순서가 없기 때문에 특정 클래스 레이블에 할당한 **숫자는 의미가 없다.**

1. ```enumerate```를 사용하여 클래스 레이블을 0부터 할당한다.<br>
(```enumerate```는 반복 가능한 객체(문자열, 리스트, 넘파이 배열 등)를 입력으로 받아 인덱스와 값의 튜플을 차례대로 반환하는 파이썬 내장 함수이다.)

In [28]:
import numpy as np
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

2. 매핑 딕셔너리를 사용하여 클래스 레이블을 정수로 변환한다.

In [29]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


---
####원본 문자열로 바꾸는 방법

1. 다음과 같이 **매핑 딕셔너리의 키-값 쌍을 뒤집**는다.

In [30]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


2. 사이킷런에 구현된 **```LabelEncoder```를 사용**한다.<br>
(```LabelEncoder``` 객체의 ```classes_```속성에 각 클래스의 레이블이 저장되어 있다.)

In [31]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

    - fit_transform 메소드는 fit 메소드와 transform 메소드를 합쳐 놓은 단축 메소드이다.

3. **```inverse_transform``` 메소드를 사용**한다.

In [32]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

## 4.2.3 순서가 없는 특성에 원-핫 인코딩 적용

순서가 없는 ```color``` 열에도 ```LabelEncoder```를 사용하여 문자열 레이블을 정수로 바꿀 수 있다.

In [33]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

- blue = 0
- green = 1
- red = 2
---

### LabelEncoder

위의 코드에서 ```color``` 열만 추출해서 주입한 이유는 **```LabelEncoder```는 입력 데이터로 1차원 배열을 받기 때문**이다.<br>

1차원 배열을 받는 이유는 **타깃 레이블을 인코딩하기 위해** 만들어진 클래스이기 때문이다.

### OrdinalEncoder & ColumnTransformer

```OrdinalEncoder```와 ```ColumnTransformer```를 함께 사용하면 **여러 개의 열을 한 번에 정수로 변환**할 수 있다.
- **```OrdinalEncoder```**: 범주형 데이터를 정수로 인코딩하는 클래스

- **```ColumnTransformer```**: pandas 데이터프레임의 열마다 다른 변환을 적용하도록 도와주는 클래스

#### ```ColumnTransformer``` 클래스

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype=np.int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

```ColumnTransformer```는 첫 번째 매개변수로 트랜스포머(transformer)의 리스트를 받는다.<br>

    def __init__(transformers, remainder='drop', sparse_threshold=0.3, n_jobs=None, transformer_weights=None, verbose=False)

**트랜스포머(transformer)**: 이름, 변환기, 변환할 열의 리스트로 이루어진 튜플
- 이름: ```ord_enc```<br>

- 변환기: ```OrdinalEncoder``` 객체<br>

- 변환할 열: ```color```

<br>

위 코드의 결과는 ```color``` 열이 정수 값으로 변환되었다.<br><br>


```ColumnTransformer```에 사용한 변환기는 ```named_transformers_``` 속성에서 앞서 지정한 ord_enc 이름으로 참조할 수 있다.<br>

정수로 인코딩된 값을 **다시 문자열로 변환**하려면 ```inverse_transform``` 메소드를 호출하면 된다.

In [35]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

#### ```OrdinalEncoder``` 클래스

    def __init__(categories='auto', dtype=np.float64)
**```dtype```** 매개변수:
- 기본값: **```np.float64```**, 실수로 인코딩한다.
- **정수**로 인코딩하고 싶으면 **```np.int```를 지정**하면 된다.

<br>

**```categories```** 매개변수:
- 기본값: ```auto```, 훈련 데이터셋에서 **자동으로 범주를 인식**한다.
- ```categories``` 매개변수에 **직접 범주 리스트를 전달**할 수 있다.
- 인식된 범주는 **```categories_``` 속성에 저장**된다.
